# SQL запросы к базе данных  

Компания приобрела крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных, в которой храниться информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель исследования:**  
Проанализировать базу данных, для последующего формирования ценностного предложения для нового продукта.  

**Задачи:**  
- Посчитать какое количество книг вышло после 1 января 2000 года
- Для каждой книги посчитать количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключить брошюры)
- Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  

**Описание данных:**   

**Таблица** `books` (содержит данные о книгах):
- *book_id* - идентификатор книги
- *author_id* - идентификатор автора
- *title* - название книги
- *num_pages* - количество страниц
- *publication_date* - дата публикации книги
- *publisher_id* - идентификатор издателя  

**Таблица** `authors` (содержит данные об авторах):
- *author_id* - идентификатор автора
- *author* - имя автора  

**Таблица** `publishers` (содержит данные об издательствах):
- *publisher_id* - идентификатор издательства
- *publisher* - название издательства  

**Таблица** `ratings` (содержит данные о пользовательских оценках книг):
- *rating_id* - идентификатор оценки
- *book_id* - идентификатор книги 
- *username* - имя пользователя, оставившего оценку  
- *rating* - оценка книги  

**Таблица** `reviews` (содержит данные о пользовательских обзорах на книги):
- *review_id* - идентификатор обзора
- *book_id* - идентификатор книги 
- *username* - имя пользователя, написавшего обзор  
- *text* - текст обзора  

In [1]:
# !pip install psycopg2


# импортируем необходимые библиотеки

import pandas as pd
import sqlalchemy as sa

In [13]:
# устанавливаем параметры

db_config = {
    'user': , # имя пользователя
    'pwd': ,  # пароль
    'host': , # хост
    'port': , # порт
    'db':     # база данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор

engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

Исследуем таблицы.  
Выведем первые строки и посчитаем количество строк в каждой таблице.

In [3]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print(f'\n\033[7;30m таблица:  {table}\033[0m\n')
    display(pd.read_sql_query(f'''select * from {table} limit 5''', engine))
    print('\n\033[1;4;30m количество строк в таблице: \033[0m')
    display(pd.read_sql_query(f'''select count(*) from {table}''', engine))


 таблица:  books



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



 количество строк в таблице: 


,count
0,1000



 таблица:  authors



...

,count
0,6456



 таблица:  reviews



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



 количество строк в таблице: 


,count
0,2793


База данных состоит из пяти таблиц, которые хранят в себе данные о книгах, их авторах, оценках и обзорах и об издательствах.  
Всего представлена информация о 1000 книг, 636 авторах, 340 издательств, более 6 тысяч оценок и почти три тысячи обзоров на книги 

**Запрос 1**  
Посчитать какое количество книг вышло после 1 января 2000 года

In [4]:
query = '''
select count(*)
from books
where publication_date > '2000-01-01'
'''
display((pd.read_sql_query(query, engine)))

,count
0,819


819  книг было выпущено после 1 января 2000 года

**Запрос 2**  
Для каждой книги посчитать количество обзоров и среднюю оценку

In [5]:
query = '''
select b.title,
       count(distinct review_id) as reviews_count,
       avg(rating) as avg_rating
from books as b
left join reviews as r on b.book_id = r.book_id
left join ratings as rt on b.book_id = rt.book_id
group by b.book_id
order by reviews_count desc, avg_rating desc
'''
display((pd.read_sql_query(query, engine)))

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,Essential Tales and Poems,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Больше всего обзоров было у книги `Twilight` (7 обзоров), при этом средний рейтинг книги значительно ниже (3.66) относительно других книг из первой пятерки. Второе и третье место по количеству обзоров занимаю произведения о "мальчике, который выжил": `Harry Potter and the Prisoner of Azkaban` и `Harry Potter and the Chamber of Secrets`.  
Есть книги, для которых еще не написано обзоров, но выставленны пользовательские оценки.  
Посмотрим как расположились книги по среднему рейтингу. К запросу добавим еще столбец с количеством пользователей, оставивших оценку

In [6]:
query = '''
select b.title,
       count(distinct review_id) as reviews_count,
       count(distinct rating_id) as user_rating_count,
       avg(rating) as avg_rating
from books as b
left join reviews as r on b.book_id = r.book_id
left join ratings as rt on b.book_id = rt.book_id
group by b.book_id
order by avg_rating desc, user_rating_count desc
'''
display((pd.read_sql_query(query, engine)))

,title,reviews_count,user_rating_count,avg_rating
0,A Dirty Job (Grim Reaper #1),4,4,5.00
1,School's Out—Forever (Maximum Ride #2),3,4,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,3,5.00
3,In the Hand of the Goddess (Song of the Liones...,2,3,5.00
4,Women,2,2,5.00
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,4,2.25
996,Drowning Ruth,3,3,2.00
997,Junky,2,2,2.00
998,His Excellency: George Washington,2,2,2.00


По среднему рейтингу лидирует книга `A Dirty Job`, ее оценило 4 пользователz и 4 пользователя написали обзор. Также в пятерку лидеров по среднему рейтингу попали книги: `School's Out—Foreverб`, `Moneyball: The Art of Winning an Unfair Game`, `In the Hand of the Goddess`, `The Walking Dead Book One`.  
Посмотрим у какого количества книг рейтинг равен 5. 

In [7]:
query = '''
with one as (select b.title,
                    count(distinct review_id) as reviews_count,
                    count(distinct rating_id) as user_rating_count,
                    avg(rating) as avg_rating
             from books as b
             left join reviews as r on b.book_id = r.book_id
             left join ratings as rt on b.book_id = rt.book_id
             group by b.book_id
             order by avg_rating desc, user_rating_count desc)
select count(title) as cnt_max_rating
from one
where avg_rating = 5
'''
display((pd.read_sql_query(query, engine)))

,cnt_max_rating
0,44


Из тысячи книг, у 44-х рейтинг равен 5 баллам.  
И наконец посмотрим на топ-5 книг с самым большим количеством пользовательских оценок.

In [8]:
query = '''
select b.title,
       count(distinct rating_id) as rating_count
from books as b
left join ratings as r on b.book_id = r.book_id
group by b.book_id
order by rating_count desc
limit 5
'''
display((pd.read_sql_query(query, engine)))

,title,rating_count
0,Twilight (Twilight #1),160
1,The Hobbit or There and Back Again,88
2,The Catcher in the Rye,86
3,Angels & Demons (Robert Langdon #1),84
4,Harry Potter and the Prisoner of Azkaban (Harr...,82


Снова лидирует произведение `Twilight`, на данную книгу было написано 7 обзоров, а пользователи оценили 160 раз. В два раза меньше была оценена книга `The Hobbit or There and Back Again` - 88 раз.

**Запрос 3**  
Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключить брошюры)

In [9]:
query = '''
select publisher, count(book_id) as total_books
from publishers as p
join books as b on p.publisher_id = b.publisher_id
where num_pages > 50
group by publisher
order by total_books desc
limit 5
'''
display((pd.read_sql_query(query, engine)))

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Больше всего книг было выпущено издательством `Penguin Books` - 42 книги, на втором месте издательство `Vintage` - 31 книга.

**Запрос 4**  
Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)

In [10]:
query = '''
select author, round(avg(rating),4) as rating
from books as b
join ratings as r on b.book_id = r.book_id
join authors as a on b.author_id = a.author_id
where b.book_id in (select r.book_id
                    from ratings as r
                    group by book_id
                    having count (*) >= 50)
group by a.author_id
order by rating desc
limit 5
'''
display((pd.read_sql_query(query, engine)))

,author,rating
0,J.K. Rowling/Mary GrandPré,4.2871
1,Markus Zusak/Cao Xuân Việt Khương,4.2642
2,J.R.R. Tolkien,4.2469
3,Louisa May Alcott,4.1923
4,Rick Riordan,4.0806


По значению среднего рейтинга книг лидирует автор `J.K. Rowling/Mary GrandPré` - 4.29. На втором и третьем местах, с совсем небольшим отрывом, авторы `Markus Zusak/Cao Xuân Việt Khương` и `J.R.R. Tolkien` ~ 4.26.

**Запрос 5**  
Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''
select avg(reviews_count) as avg_reviews
from (select username,
             count(review_id) as reviews_count
      from reviews
      where username in (select username
                        from ratings
                        group by username
                        having count(rating_id) > 48)
      group by username) as user_reviews
'''
display((pd.read_sql_query(query, engine)))

,avg_reviews
0,24.0


Пользователи, поставившие более 48ми оценок, в среднем написали 24 обзора на книги

**Вывод**  

- Всего в сервисе представлено 1000 книг от 636 авторов
- 819 книг было выпущено после 1 января 2000 года
- Больше всего обзоров написано на книгу `Twilight` - 7 обзоров (данная книга лидирует и по количеству пользовательских оценок - 160 раз), второе и третье место по количеству обзоров занимают произведения J.K. Rowling `Harry Potter and the Prisoner of Azkaban` и `Harry Potter and the Chamber of Secrets` (6 обзоров).
- Из тысячи книг, у 44-х рейтинг равен 5 баллам.
- Больше всего книг было выпущено издательством `Penguin Books` - 42 книги, на втором месте издательство `Vintage` - 31 книга.
- По значению среднего рейтинга книг лидирует автор `J.K. Rowling/Mary GrandPré` - 4.28. На втором и третьем местах, с совсем небольшим отрывом, авторы `Markus Zusak/Cao Xuân Việt Khương` и `J.R.R. Tolkien` - 4.26
- В среднем, самые активные пользователи, написали 24 обзора на книги.